In [1]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display, HTML
import msgpack as mp
import msgpack_numpy as mpn
import base64
from datetime import datetime
import os
from tqdm import tqdm
from datetime import timedelta
import keyboard

In [2]:
# base_pth = "C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 08-02-22_13-24-38_725"
# base_pth = r"C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 04-02-22_14-03-00_359"
base_pth = r"C:\mira\splitVideos\SAMXXXXXXXXU010120000000XXXXXXXXX\Session 10-02-22_09-14-15_436"
_prm = os.listdir(base_pth)

for i in _prm:
    if i.startswith("PARAMS"):
        _p_fname = i
prm_file = base_pth + "\\" + _p_fname
prm_file


p = open(prm_file, "rb")
unpacker = mp.Unpacker(p, object_hook=mpn.decode)
prm = []
for unpacked in unpacker:
    prm.append(unpacked)
prm.pop(0)
prm.pop(0)


video_time = []
for i in prm:
    video_time.append(datetime.strptime(i[0], '%Y-%m-%d %H:%M:%S.%f'))
v_start = video_time[0]
v_start


"""for df"""
video_time_df = []
for i in prm:
    video_time_df.append(i[0])



In [3]:
video_data = cv2.VideoCapture(base_pth+"//Video.avi")

_i = 0
frames = []
while video_data.isOpened:
    
    ret, frame = video_data.read()
    if not ret:
        break
    frames.append(frame)
    frame = cv2.flip(frame, 1)
    vid_diff = video_time[_i] - video_time[0] 
    _vid_diff = vid_diff.total_seconds()
    _i += 1
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # org
    org = (50, 50)
    
    # fontScale
    fontScale = 1
    
    # Blue color in BGR
    color = (255, 0, 0)
    
    # Line thickness of 2 px
    thickness = 2
    
    # Using cv2.putText() method
    image = cv2.putText(frame, str(_vid_diff), org, font, 
                    fontScale, color, thickness, cv2.LINE_AA)
    if ret:
        cv2.imshow("",frame)
        # plt.show()
        
    if cv2.waitKey(25) & 0xFF == ord('q'):
          break
    
    
    
    
cv2.destroyAllWindows()
        

In [4]:
imu_1 = pd.read_csv(base_pth+"//1_imu.csv")
imu_2 = pd.read_csv(base_pth+"//2_imu.csv")

In [5]:
def datetime_to_s(df, c_name, sys_t=None, offset=0):
    """this function changes datetime string to seconds"""
    inf = datetime.strptime(df[c_name][0], '%Y-%m-%d %H:%M:%S.%f')
    dif_list = []
    for idx, val in enumerate(df[c_name]):
        _val = datetime.strptime(val, '%Y-%m-%d %H:%M:%S.%f')
        if sys_t:
            # diff = (_val - inf) - sys_t
            diff = _val - sys_t

        else:
            diff = _val - inf
            
        _diff = diff.total_seconds()
        # df["seconds"].loc[idx] = _diff
        dif_list.append(_diff)
    df["sys_sec"] = dif_list
    return df

In [6]:
# plt.plot(imu_1["imu_time"], imu_1["ax"])
imu_1 = datetime_to_s(imu_1, "sys_time", v_start)
imu_2 = datetime_to_s(imu_2, "sys_time", v_start)

"""for df"""

imu_1df = pd.read_csv(base_pth+"//1_imu.csv", parse_dates=['sys_time'], index_col='sys_time')
imu_1df.index.name = "time"
imu_2df = pd.read_csv(base_pth+"//2_imu.csv", parse_dates=['sys_time'], index_col='sys_time')
imu_2df.index.name = "time"
vid_df = pd.DataFrame(video_time_df, columns=["time"])
vid_df = pd.to_datetime(vid_df["time"])
# imu_1df

In [7]:
# plt.plot(imu_1["sys_sec"], imu_1["az"])
# imu_1df
vid_df

0     2022-02-10 09:14:54.151231
1     2022-02-10 09:14:54.645738
2     2022-02-10 09:14:54.712061
3     2022-02-10 09:14:54.779997
4     2022-02-10 09:14:54.847640
                 ...            
993   2022-02-10 09:16:08.373098
994   2022-02-10 09:16:08.440099
995   2022-02-10 09:16:08.506782
996   2022-02-10 09:16:08.573036
997   2022-02-10 09:16:08.640148
Name: time, Length: 998, dtype: datetime64[ns]

In [8]:
imutime = [] 
for t in tqdm(vid_df[:len(frames)-1]):
    
    # print(t)
    imutime.append(imu_1df.index.unique()[imu_1df.index.unique().get_loc(t, method='nearest')])
grps = []
for t in tqdm(imutime):
    grps.append(imu_1df.between_time((t - timedelta(seconds=2)).time(), (t + timedelta(seconds=1)).time()))

100%|██████████| 997/997 [00:00<00:00, 5085.99it/s]


In [9]:
imutime2 = [] 
for t in tqdm(vid_df[:len(frames)-1]):
    
    # print(t)
    imutime2.append(imu_2df.index.unique()[imu_2df.index.unique().get_loc(t, method='nearest')])
grps2 = []
for t in tqdm(imutime2):
    grps2.append(imu_2df.between_time((t - timedelta(seconds=2)).time(), (t + timedelta(seconds=1)).time()))

100%|██████████| 997/997 [00:00<00:00, 5164.60it/s]


In [10]:
imu_1df.index.unique()[imu_1df.index.unique().get_loc(t, method='nearest')]

Timestamp('2022-02-10 09:16:06.064188')

In [11]:
# imu_1df.index.get_loc("2022-02-04 14:04:38.745642", method="nearest")

# imu_1df.index[0]
%matplotlib qt

In [12]:
grps

[                              imu_time        ax        ay        az  \
 time                                                                   
 2022-02-10 09:15:07.680781  457.790501  0.014404 -0.027954  1.008301   
 2022-02-10 09:15:07.680781  457.809273  0.012573 -0.026611  1.010498   
 2022-02-10 09:15:07.681783  457.818459  0.014282 -0.026855  1.009521   
 2022-02-10 09:15:07.685780  457.827642  0.014160 -0.026001  1.009521   
 2022-02-10 09:15:07.685780  457.836880  0.012939 -0.025879  1.009521   
 ...                                ...       ...       ...       ...   
 2022-02-10 09:15:08.644783  458.744145  0.014526 -0.027100  1.009644   
 2022-02-10 09:15:08.646783  458.753330  0.013184 -0.026733  1.009521   
 2022-02-10 09:15:08.648781  458.762515  0.014160 -0.025635  1.010620   
 2022-02-10 09:15:08.648781  458.772102  0.013672 -0.026611  1.010498   
 2022-02-10 09:15:08.650783  458.781175  0.013672 -0.027832  1.009033   
 
                                   gx        gy  

In [13]:
start_at = 300
i=0

for frame, t, dat in zip(frames[start_at+1:], imutime[start_at:], grps[start_at:]):
    dat
    if keyboard.is_pressed('q'):
        plt.close()
        break
    else:
        plt.clf()
        plt.subplot(121)
        plt.title(i)
        plt.imshow(frame)
        plt.subplot(322)
        plt.title(t)
        plt.plot(list(dat.ax))
        plt.axvline(t, color='r')
        plt.ylim(imu_1df.ax.min(), imu_1df.ay.max())
        plt.subplot(324)
        plt.plot(list(dat.ay))
        plt.axvline(t, color='r')
        plt.ylim(imu_1df.ay.min(), imu_1df.ay.max())
        plt.subplot(326)
        plt.plot(list(dat.az))
        plt.axvline(t, color='r')
        plt.ylim(imu_1df.az.min(), imu_1df.az.max())
        i+=1
        plt.pause(0.005)
    break

In [14]:
plt.plot(imutime2)

In [15]:
plt.plot(imu_1df["ax"])

In [16]:
# plt.plot(t, dat.ax)

In [18]:
plt.plot(imu_1df["imu_time"].diff())